In [ ]:
from instagrapi import Client
from instagrapi.exceptions import UserNotFound
import configparser, time, shutil, os, re, unicodedata, sys 
from datetime import datetime


config = configparser.ConfigParser()

config_path = "config.ini"
if not os.path.exists(config_path):
    raise FileNotFoundError(f"Error: The file '{config_path}' does not exist.")
else:
    config.read(config_path, encoding="utf-8")

USERNAME = config["instagram"]["username"]
PASSWORD = config["instagram"]["password"]

cl = Client()
cl.login(USERNAME, PASSWORD)

In [ ]:
def get_accounts_num(target_account, config):

    # 정규식으로 계정명과 숫자 추출
    match = re.match(r"(https?://www\.instagram\.com/)?([\w\.]+)(?:\s*//\s*(\d+))?", target_account)

    target_account = match.group(2)  # 계정명
    target_post_count = int(match.group(3)) if match.group(3) else int(config["instagram"]["DEFAULT_NUM"])

    return target_account, target_post_count


def get_latest_datetime_from_files(folder_path):
    date_pattern = re.compile(r"\[(\d{4}-\d{2}-\d{2})\]")  # 날짜 패턴 [YYYY-MM-DD]
    latest_datetime = None

    for file_name in os.listdir(folder_path):
        match = date_pattern.search(file_name)
        if match:
            file_datetime = datetime.strptime(match.group(1), "%Y-%m-%d")
            if not latest_datetime or file_datetime > latest_datetime:
                latest_datetime = file_datetime

    return latest_datetime

while True:
    config.read(config_path, encoding="utf-8")

    ACCOUNTS_LIST = config["instagram"]["accounts_list"].splitlines()
    ACCOUNTS_LIST = [url.strip() for url in ACCOUNTS_LIST if url and url.strip()]

    include_stories = True if config["instagram"]["include_story"].strip().lower() == "true" else False
    overwrite = True if config["instagram"]["overwrite"].strip().lower() == "true" else False


    # ☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★
    for target_account in ACCOUNTS_LIST:

        target_account, target_post_count = get_accounts_num(target_account, config)

        try:
            user_info = cl.user_info_by_username(target_account)
        except UserNotFound:
            print(f"User not found. Please check the username. -> {target_account}")
            continue
        user_id, user_name = user_info.pk, user_info.full_name

        download_folder = f"{user_name} (insta_{target_account})"
        download_folder = os.path.join(config["instagram"]["default_path"], download_folder)
        download_folder = re.sub(r'[*?"<>|]', "", download_folder.replace("|", "｜"))
        download_folder = unicodedata.normalize("NFC", download_folder)

        if not os.path.exists(download_folder):  # 폴더 생성
            os.makedirs(download_folder)

        if include_stories:  # 스토리 다운로드
            stories = cl.user_stories(user_id)
            if not isinstance(stories, list):
                stories = [stories]

            for i_story, story in enumerate(stories):
                cl.story_download(stories[i_story].pk, filename=f"stories_{stories[i_story].id}", folder=download_folder)

        if not overwrite:
            files=os.listdir(download_folder)

        medias = cl.user_medias(user_id, target_post_count)  # 게시글 정보 수집
        medias = medias[:3] + medias[3:][::-1] # 오래된것부터 먼저 다운로드 하기 위해 정렬
        
        i,skip_file = 0,False
        while i < len(medias):  # 다운로드중 timeout 발생시 다시 다운받게하려고 for문 대신 while문 씀
            media = medias[i]
            print(f"\r{user_name} (insta_{target_account}) \t {i+1}/{len(medias)} - {(i+1)/len(medias) * 100:.1f}%", end="")
            
            if not overwrite:
                for file in files:
                    if media.code in file:
                        skip_file=True
                        break
            if skip_file:
                i=i+1
                continue

            download_methods = {1: cl.photo_download, 2: cl.video_download, 8: cl.album_download}  # Photo  # Video  # Album
            try:
                return_paths = download_methods.get(media.media_type, lambda *_: None)(media.pk, download_folder)
            except:
                time.sleep(60)
                continue
            if not isinstance(return_paths, list):
                return_paths = [return_paths]
            for ii, path in enumerate(return_paths):
                prefix, ext = str(path).rsplit("\\", 1)[0], str(path).rsplit(".", 1)[1]
                new_path = os.path.join(prefix, (f"[{media.taken_at.date()}] {media.code}_p{ii}.{ext}"))
                shutil.move(str(path), new_path)
            i = i + 1
        print("")